In [1]:
import sys
import os

current_directory = os.getcwd()
path = os.path.dirname(current_directory)
sys.path.append(path)
from Utils import *

%matplotlib widget
from ipywidgets import interact, interactive, widgets
from matplotlib.patches import Rectangle, Circle, Arrow

In [3]:
import glob
import os

import tempfile
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from typing import Optional, Any, Mapping, Hashable

import monai
from monai.config import print_config
from monai.utils import first
from monai.config import KeysCollection
from monai.data import Dataset, ArrayDataset, create_test_image_3d, DataLoader
from monai.transforms import (
    Transform,
    Compose,
    EnsureChannelFirst,
    LoadImage,
    Orientation,
)
# print_config()

# Load images
srcdir = os.getcwd()

print('Reading labeled images from: ' + srcdir)
images = sorted(glob.glob(os.path.join(srcdir, "*_M.nii.gz")))
labels = sorted(glob.glob(os.path.join(srcdir, "*_needle_label.nii.gz")))

print(images)

# Create dictionary
train_files = [
    {'image': image_name, 'label': label_name}
    for image_name, label_name in zip(images, labels)
]
img_file = train_files[0]['image']
label_file = train_files[0]['label']
fn_keys = ('image', 'label')  # filename keys for image and seg files

transfTest = Compose([
                        LoadImage(image_only=True),
                        EnsureChannelFirst(channel_dim='no_channel'),
                        Orientation(axcodes='PIL'),
                    ])
img = transfTest(img_file)
img_arr = np.array(img) # Numpy array image: [channel, slice, horizontal, vertical]
print(np.shape(img_arr))
show_array(img[0,:,:,:])



Reading labeled images from: /Users/pl771/Devel/MRINeedleSegmentation/TestingNotebook
['/Users/pl771/Devel/MRINeedleSegmentation/TestingNotebook/Test_M.nii.gz']
(1, 3, 192, 192)


interactive(children=(IntSlider(value=1, description='z', max=2), Output()), _dom_classes=('widget-interact',)…